### Pinecone Vector DB

In [1]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone import ServerlessSpec
from langchain_openai import OpenAIEmbeddings

/home/aniruddha/Projects/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api_key = os.getenv("PINECONE_TOKEN")
pc = Pinecone(api_key=api_key)

In [4]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1024
)

In [6]:
index_name = "rag-udemy"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [7]:
index

In [8]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [9]:
vector_store

In [10]:
from langchain_core.documents import Document


document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [12]:
vector_store.add_documents(documents)

['4e420b73-2a33-4b97-b14d-7b27da5e1c36',
 '01a7bb93-15e6-44f3-bf86-2c786ce4ff26',
 '67dbfa43-5c70-41fd-974b-83288dd53ac7',
 '5c9ebae0-2eb8-4fc2-ad3f-47eca5599f3b',
 '7abaec86-513a-4490-ab1d-3c4bb95638d3',
 '0ef3c022-343d-4175-b673-0d30ece5454f',
 '27159ef2-f126-4b5e-b4ce-27a7c2feaffb',
 '9e4d2684-16b3-4446-b21f-f8be2dba98f5',
 '48aee4b7-385d-4335-aef4-ea3e87a0061c',
 'f02bfde9-555a-4e63-a7b4-2c4e3040aa3a']

In [13]:
### Query Directly
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [14]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.572732] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [ ]:
### Retriever
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.4},
)
retriever.invoke("Stealing from the bank is a crime", filter={"source": "news"})